In [1]:
import re

file_location = '../Data/DTU Fact Check/Undertekster/'
files = ['{:s}{:d}/Undertekster_ Debatten.html'.format(file_location,i) for i in range(1,11)]

In [2]:
!ls ../Data/DTU\ Fact\ Check/Undertekster/2

Undertekster_ Debatten_files  Undertekster_ Debatten.html


In [3]:
# File locations
files

['../Data/DTU Fact Check/Undertekster/1/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/2/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/3/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/4/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/5/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/6/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/7/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/8/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/9/Undertekster_ Debatten.html',
 '../Data/DTU Fact Check/Undertekster/10/Undertekster_ Debatten.html']

In [4]:
def getHtmlSentences(file_loc):
    subtitle_file = open(file_loc,'r')
    doc = subtitle_file.read()

    subtitle_file.close()

    # Find part of the html which contain the subtitles
    p_subs = re.compile('category="Undertekster"[\S\W<>]+<script src="./Undertekster_ Debatten_files/dash.all.min.js.download"></script>')
    match = re.search(p_subs,doc)
    doc_subs = match.group()

    #Split into the sentences
    sentences = doc_subs.split('<span class="digits ng-binding">')
    sentences = [sentences[i] for i in range(1,len(sentences))]

    print('\tProgram has {:d} sentences'.format(len(sentences)))
    return sentences

program_sentences = getHtmlSentences(files[0])

	Program has 766 sentences


In [5]:
def getTimeAndText(text):
    p_time = re.compile('[\d:]+')

    p_text = re.compile('ma-highlight="[\w?%!:; ,.\d-]+')
    a = re.finditer(p_text,text)

    s = ''
    for i in a:
        s=s+' '+(i.group()[14:])
    
    s = re.sub(' +',' ',s) # Removes repeating whitespaces
    s=s[1:] # Removes first white space
    
    nums = re.finditer(p_time,text)
    nums = [n.group() for n in nums]
    
    time_start = nums[0]
    time_end = nums[1]
    
    return s, time_start, time_end

getTimeAndText(program_sentences[0])

('Er det stadig danske politikere, der bestemmer over samfundets udvikling?',
 '20:00:01:01',
 '20:00:05:20')

In [6]:
def getCleanedSentences(sentences):
    program = []
    start_times = []
    end_times = []

    for sen in sentences:#[5:]:
        text, start, end = getTimeAndText(sen)
        #print(text)
        #print('\n')

        if len(program) is 0:
            program.append(text)
            start_times.append(start)
            end_times.append(end)
        elif len(text)>0:
            try:

                if text[0] is '-':
                    last_text = program.pop()
                    s = last_text+text
                    s = re.sub('-+','',s)
                    s = re.sub(' +',' ',s)

                    program.append(s)

                    end_times.pop()
                    end_times.append(end)

                else:
                    program.append(text)
                    start_times.append(start)
                    end_times.append(end)
                    
            except Exception as e:
                print('Woops.. ')
                print('"',text,'"')
                print(len(text))
    
    print('\tProgram has {:d} cleaned sentences'.format(len(program)))
    return dict(zip(['sentences','start time','end time'],[program, start_times, end_times]))

cleaned_sentences = getCleanedSentences(program_sentences)

	Program has 516 cleaned sentences


In [7]:
total_sent = 0
all_programs = dict()
for i in range(len(files)):
    print('Program',i+1)
    program_sentences = getHtmlSentences(files[i])
    all_programs['program'+str(i+1)] = getCleanedSentences(program_sentences)
    total_sent += len(all_programs['program'+str(i+1)]['sentences'])

Program 1
	Program has 766 sentences
	Program has 516 cleaned sentences
Program 2
	Program has 420 sentences
	Program has 320 cleaned sentences
Program 3
	Program has 599 sentences
	Program has 442 cleaned sentences
Program 4
	Program has 485 sentences
	Program has 317 cleaned sentences
Program 5
	Program has 441 sentences
	Program has 309 cleaned sentences
Program 6
	Program has 477 sentences
	Program has 334 cleaned sentences
Program 7
	Program has 448 sentences
	Program has 307 cleaned sentences
Program 8
	Program has 566 sentences
	Program has 406 cleaned sentences
Program 9
	Program has 427 sentences
	Program has 307 cleaned sentences
Program 10
	Program has 345 sentences
	Program has 231 cleaned sentences


In [8]:
print('Samlet antal sætninger: {:d}'.format(total_sent))

Samlet antal sætninger: 3489


# Output til Philip

In [9]:
def export_debatten_program(program, program_id, 
                            write_loc = '../Results/DeepFact/'):
    sentenc_id = 1

    with open(write_loc+'program'+str(program_id), 'w') as f:
        f.write('<span id="program'+str(program_id)+'">')

        for s in program['sentences']:
            f.write('\t<p id="'+str(sentenc_id)+'"> '+s+' </p>')
            sentenc_id+=1

        f.write('</span>')
    print('Sucessfully wrote program'+str(program_id))
        
export_debatten_program(all_programs['program1'],1)

Sucessfully wrote program1


In [10]:
!ls ../Results/DeepFact/

program1   program2  program4  program6  program8
program10  program3  program5  program7  program9


In [11]:
for i in range(len(all_programs)):
    export_debatten_program(all_programs['program'+str(i+1)],i+1)

Sucessfully wrote program1
Sucessfully wrote program2
Sucessfully wrote program3
Sucessfully wrote program4
Sucessfully wrote program5
Sucessfully wrote program6
Sucessfully wrote program7
Sucessfully wrote program8
Sucessfully wrote program9
Sucessfully wrote program10


In [12]:
!ls -l -h ../Results/DeepFact/

total 348K
-rw-r--r-- 1 jehi unixusers 51K Oct  2 16:36 program1
-rw-r--r-- 1 jehi unixusers 23K Oct  2 16:36 program10
-rw-r--r-- 1 jehi unixusers 30K Oct  2 16:36 program2
-rw-r--r-- 1 jehi unixusers 39K Oct  2 16:36 program3
-rw-r--r-- 1 jehi unixusers 32K Oct  2 16:36 program4
-rw-r--r-- 1 jehi unixusers 30K Oct  2 16:36 program5
-rw-r--r-- 1 jehi unixusers 32K Oct  2 16:36 program6
-rw-r--r-- 1 jehi unixusers 31K Oct  2 16:36 program7
-rw-r--r-- 1 jehi unixusers 39K Oct  2 16:36 program8
-rw-r--r-- 1 jehi unixusers 30K Oct  2 16:36 program9
